#Requirments :

In [3]:
!pip install nltk

In [33]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
df1 = pd.read_csv('/content/drive/MyDrive/Buiness_Model/BuinessModel/Hostel_data/hostel_detail_data.csv',
                   sep=';',
                  quoting=csv.QUOTE_ALL,
                  escapechar='\\',
                  on_bad_lines='skip')

In [6]:
df1.head(50)

,id_hostel,name,currency,type,maxNumberOfGuestsPerBooking,isFeatured,latestCheckOut,rating,totalRatings,starRating,...,payments.payNow,cancellationPolicy.depositOnly.description,cancellationPolicy.depositOnly.numberOfDays,cancellationPolicy.nonRefundable.description,cancellationPolicy.freeCancellation.description,freeCancellation.isAvailable,freeCancellation.daysInAdvance,groupInformation.groupMaxPax,taxInfo.included,taxInfo.taxes
0,177051,NattyDog,EUR,HOSTEL,8,False,NaN,NaN,10,0,...,False,This property has a 2 day cancellation policy....,2,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,3.0,NaN,True,[]
1,177048,Hostel MAK Sarajevo,EUR,GUESTHOUSE,8,False,NaN,NaN,1,0,...,False,This property has a 3 day cancellation policy....,3,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,4.0,24.0,True,[]
2,177041,StarVillas Apartments,EUR,APARTMENT,8,False,NaN,NaN,5,0,...,False,This property has a 3 day cancellation policy....,3,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,4.0,NaN,True,[]
3,177037,Andrews Apartments,EUR,APARTMENT,8,False,NaN,NaN,0,0,...,False,This property has a 3 day cancellation policy....,3,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,4.0,27.0,True,[]
4,177035,Hostal El Parque,USD,GUESTHOUSE,8,False,NaN,NaN,4,0,...,False,This property has a 3 day cancellation policy....,3,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,4.0,NaN,True,[]
5,177034,Hostel Old Wall,USD,HOSTEL,8,False,NaN,NaN,5,0,...,False,This property has a 3 day cancellation policy....,3,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,4.0,NaN,True,[]
6,177030,Estancia la Serena,ARS,HOSTEL,8,False,NaN,NaN,1,0,...,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,2.0,NaN,True,[]
7,177024,Milos Studios,EUR,HOTEL,8,False,NaN,NaN,2,0,...,False,This property has a 3 day cancellation policy....,3,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,4.0,NaN,True,[]
8,177025,Isis Hostel 2,USD,HOTEL,36,False,NaN,NaN,0,0,...,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,2.0,NaN,True,[]
9,102883,Riad Dar Attika,MAD,HOTEL,8,False,NaN,NaN,59,0,...,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...","Your booking can be cancelled for free, if you...",True,2.0,NaN,False,"[{'type': 'per person per night', 'amount': 26..."


In [7]:
df1.shape

(41642, 36)

In [8]:
df2 = pd.read_csv('/content/drive/MyDrive/Buiness_Model/BuinessModel/Hostel_data/hostel_review_data.csv',
                   sep=';',
                  quoting=csv.QUOTE_ALL,
                  escapechar='\\',
                  on_bad_lines='skip')

<ipython-input-8-3fc0fedb6438>:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/drive/MyDrive/Buiness_Model/BuinessModel/Hostel_data/hostel_review_data.csv',


In [9]:
df2.head()

,id_review,id_hostel,id_user,review_date,review_comment,review_owner_comment,user_gender_code,user_gender_code_2,user_age,user_trip_code,...,user_nationality_name,user_number_reviews,review_rating,review_safety_rate,review_location_rate,review_staff_rate,review_atmosphere_rate,review_cleanliness_rate,review_facilities_rate,review_rating_overall
0,13601903,17,33235245,2021-07-26,Loved this place so much. Amazing location and...,Thank you sooo much! \nHope to see here with u...,FEMALE,Female,18-24,RTWTRIP,...,Netherlands,3.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,13603073,17,12275191,2021-07-23,Ver nice place for a short stay to visit Valen...,NaN,MALE,NaN,25-30,WEEKENDAWAY,...,France,6.0,80.0,100.0,80.0,100.0,80.0,80.0,60.0,83.0
2,13597104,17,9747771,2021-07-21,The staff were friendly and very helpful! Rea...,How happy we are to know that we made sure you...,MALE,Male,41+,REGULARVACATION,...,USA,15.0,100.0,100.0,100.0,100.0,80.0,80.0,80.0,91.0
3,13614587,17,33233596,2021-07-20,Will definitely come back :),NaN,COUPLE,Male,25-30,OTHER,...,Germany,27.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,13599778,17,33196463,2021-07-18,This hostel was great. The staff was super nic...,"Thanks for your comment,\nOur hostel is great ...",MALE,Male,18-24,GAPYEAR,...,Netherlands,2.0,100.0,100.0,100.0,100.0,80.0,100.0,100.0,97.0


In [10]:
df2.shape

(934209, 21)

In [11]:
df1.isnull().sum()

,0
id_hostel,0
name,0
currency,0
type,0
maxNumberOfGuestsPerBooking,0
isFeatured,0
latestCheckOut,21070
rating,41642
totalRatings,0
starRating,0


In [12]:
df1 = df1.drop(['region.id' , 'region.name' , 'cancellationPolicy.freeCancellation.description' ,
                'freeCancellation.daysInAdvance' , 'groupInformation.groupMaxPax' , 'taxInfo.included' ,
                 'taxInfo.taxes' , 'address2'   , 'rating' , 'starRating',
                 'latestCheckOut' ] , axis=1)

In [13]:
df_n = df1.dropna(axis=0)
df_n['id_hostel'] = df_n['id_hostel'].astype(int)
df_n.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41416 entries, 0 to 41641
Data columns (total 25 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id_hostel                                     41416 non-null  int64  
 1   name                                          41416 non-null  object 
 2   currency                                      41416 non-null  object 
 3   type                                          41416 non-null  object 
 4   maxNumberOfGuestsPerBooking                   41416 non-null  int64  
 5   isFeatured                                    41416 non-null  bool   
 6   totalRatings                                  41416 non-null  int64  
 7   facilities                                    41416 non-null  object 
 8   description                                   41416 non-null  object 
 9   latitude                                      41416 non-null  floa

<ipython-input-13-03c30de6e84d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n['id_hostel'] = df_n['id_hostel'].astype(int)


In [14]:
df2 = df2.drop(['user_gender_code_2'  ,'review_owner_comment' ,'user_gender_code_2',
                ] , axis = 1)

In [15]:
df_rev=df2.dropna(axis=0)

#Df-2
* Review


In [16]:
df_rev.head()

,id_review,id_hostel,id_user,review_date,review_comment,user_gender_code,user_age,user_trip_code,user_nationality_code,user_nationality_name,user_number_reviews,review_rating,review_safety_rate,review_location_rate,review_staff_rate,review_atmosphere_rate,review_cleanliness_rate,review_facilities_rate,review_rating_overall
0,13601903,17,33235245,2021-07-26,Loved this place so much. Amazing location and...,FEMALE,18-24,RTWTRIP,NL,Netherlands,3.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,13603073,17,12275191,2021-07-23,Ver nice place for a short stay to visit Valen...,MALE,25-30,WEEKENDAWAY,FR,France,6.0,80.0,100.0,80.0,100.0,80.0,80.0,60.0,83.0
2,13597104,17,9747771,2021-07-21,The staff were friendly and very helpful! Rea...,MALE,41+,REGULARVACATION,US,USA,15.0,100.0,100.0,100.0,100.0,80.0,80.0,80.0,91.0
3,13614587,17,33233596,2021-07-20,Will definitely come back :),COUPLE,25-30,OTHER,DE,Germany,27.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,13599778,17,33196463,2021-07-18,This hostel was great. The staff was super nic...,MALE,18-24,GAPYEAR,NL,Netherlands,2.0,100.0,100.0,100.0,100.0,80.0,100.0,100.0,97.0


In [17]:
df_sent = df_rev[['id_hostel' , 'review_comment' , 'review_rating_overall']].copy()

In [18]:
df_sent.head()

,id_hostel,review_comment,review_rating_overall
0,17,Loved this place so much. Amazing location and...,100.0
1,17,Ver nice place for a short stay to visit Valen...,83.0
2,17,The staff were friendly and very helpful! Rea...,91.0
3,17,Will definitely come back :),100.0
4,17,This hostel was great. The staff was super nic...,97.0


In [19]:
nltk.download('vader_lexicon')
sentiment_checker=SentimentIntensityAnalyzer()
sentiments=[]
for text in df_sent['review_comment']:
    if pd.isna(text):
     sentiments.append('neutral')
     continue
    sentiment_score=sentiment_checker.polarity_scores(text)
    if(sentiment_score['pos']>sentiment_score['neg']):
      sentiments.append('positive')
    elif sentiment_score['pos']<sentiment_score['neg']:
      sentiments.append('negative')
    else:
      sentiments.append('neutral')

df_sent['sentiments']=sentiments

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [20]:
df_sent.columns

Index(['id_hostel', 'review_comment', 'review_rating_overall', 'sentiments'], dtype='object')

In [21]:
df_sent['id_hostel'] = df_sent['id_hostel'].astype(int)
df_sent.info()


<class 'pandas.core.frame.DataFrame'>
Index: 934142 entries, 0 to 934208
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id_hostel              934142 non-null  int64  
 1   review_comment         934142 non-null  object 
 2   review_rating_overall  934142 non-null  float64
 3   sentiments             934142 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 35.6+ MB


In [22]:
df_n.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41416 entries, 0 to 41641
Data columns (total 25 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id_hostel                                     41416 non-null  int64  
 1   name                                          41416 non-null  object 
 2   currency                                      41416 non-null  object 
 3   type                                          41416 non-null  object 
 4   maxNumberOfGuestsPerBooking                   41416 non-null  int64  
 5   isFeatured                                    41416 non-null  bool   
 6   totalRatings                                  41416 non-null  int64  
 7   facilities                                    41416 non-null  object 
 8   description                                   41416 non-null  object 
 9   latitude                                      41416 non-null  floa

#Adding the average rating for each of the hostel according to the hostel id

In [23]:
average_ratings = df_sent.groupby('id_hostel')['review_rating_overall'].mean().reset_index()
average_ratings.columns = ['id_hostel', 'average_rating']
average_ratings.head()

,id_hostel,average_rating
0,17,92.772549
1,19,92.576923
2,37,43.000000
3,41,100.000000
4,50,71.538462


In [24]:
df_main=pd.merge(df_n,average_ratings,on='id_hostel',how='inner')

#Adding the sentiments

In [25]:
sentiment_counts = df_sent.groupby('id_hostel')['sentiments'].value_counts().unstack(fill_value=0)
sentiment_counts.columns = ['count_positive', 'count_negative', 'count_neutral']

In [26]:
df_main = pd.merge(df_main, sentiment_counts, on='id_hostel', how='left')

In [27]:
df_main.head()

,id_hostel,name,currency,type,maxNumberOfGuestsPerBooking,isFeatured,totalRatings,facilities,description,latitude,...,city.idCountry,payments.payNow,cancellationPolicy.depositOnly.description,cancellationPolicy.depositOnly.numberOfDays,cancellationPolicy.nonRefundable.description,freeCancellation.isAvailable,average_rating,count_positive,count_negative,count_neutral
0,102883,Riad Dar Attika,MAD,HOTEL,8,False,59,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",Riad Dar Attika has kept the tradition of the ...,31.622111,...,142,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...",True,91.090909,2,3,6
1,102835,Inter-Hotel Le savoy,EUR,HOTEL,8,False,31,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",In the heart of the city of William the Conque...,49.169292,...,73,False,This property has a 2 day cancellation policy....,2,"Your booking is non-refundable, if you choose ...",True,88.142857,0,0,7
2,102833,Treasure State Hostel,USD,HOSTEL,8,False,323,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...","We are a safe, clean, and affordable hostel lo...",45.679564,...,222,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...",True,90.011111,9,4,77
3,102800,Onederz Hostel Siem Reap,USD,HOSTEL,8,False,4771,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...","If you are looking for a clean, comfortable, w...",13.356250,...,36,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...",True,95.557003,30,41,543
4,102792,Restiny Hostel,THB,HOSTEL,8,False,79,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",Restiny Hostel is a small hostel in downtown o...,13.754454,...,208,False,This property has a 1 day cancellation policy....,1,"Your booking is non-refundable, if you choose ...",True,88.846154,0,2,11


In [28]:
df_main['facilities'][0]

"[{'numericId': 1, 'name': 'Free', 'id': 'FACILITYCATEGORYFREE', 'facilities': [{'numericId': 1, 'name': 'Free Breakfast', 'id': 'BREAKFASTINCLUDED'}, {'numericId': 103, 'name': 'Free Internet Access', 'id': 'FREEINTERNETACCESS'}, {'numericId': 90, 'name': 'Free WiFi', 'id': 'FREEWIFI'}]}, {'numericId': 2, 'name': 'General', 'id': 'FACILITYCATEGORYGENERAL', 'facilities': [{'numericId': 78, 'name': 'Adaptors', 'id': 'ADAPTORS'}, {'numericId': 50, 'name': 'Air Conditioning', 'id': 'AIRCONDITIONING'}, {'numericId': 106, 'name': 'Cable TV', 'id': 'CABLETV'}, {'numericId': 91, 'name': 'Cots available', 'id': 'COTSAVAILABLE'}, {'numericId': 119, 'name': 'Hair Dryers', 'id': 'HAIRDRYERS'}, {'numericId': 51, 'name': 'Meeting Room', 'id': 'MEETINGROOM'}, {'numericId': 107, 'name': 'Outdoor Terrace', 'id': 'OUTDOORTERRACE'}, {'numericId': 116, 'name': 'Parking', 'id': 'PARKING'}, {'numericId': 111, 'name': 'Reading Light', 'id': 'READINGLIGHT'}, {'numericId': 123, 'name': 'Servizi aggiornate', '

In [29]:
df_main.columns

Index(['id_hostel', 'name', 'currency', 'type', 'maxNumberOfGuestsPerBooking',
       'isFeatured', 'totalRatings', 'facilities', 'description', 'latitude',
       'longitude', 'address1', 'directions', 'depositPercentage',
       'checkIn.startsAt', 'checkIn.endsAt', 'city.id', 'city.name',
       'city.country', 'city.idCountry', 'payments.payNow',
       'cancellationPolicy.depositOnly.description',
       'cancellationPolicy.depositOnly.numberOfDays',
       'cancellationPolicy.nonRefundable.description',
       'freeCancellation.isAvailable', 'average_rating', 'count_positive',
       'count_negative', 'count_neutral'],
      dtype='object')

In [30]:
df_main = df_main.drop(['checkIn.startsAt' , 'city.idCountry' , 'payments.payNow' ,
                       'cancellationPolicy.depositOnly.description',
                     'cancellationPolicy.depositOnly.numberOfDays',
                           'cancellationPolicy.nonRefundable.description',
                      'freeCancellation.isAvailable' , 'maxNumberOfGuestsPerBooking' ,
                        'isFeatured' , 'city.id' , 'city.name' , 'depositPercentage' , 'checkIn.endsAt' , 'city.country',
                        'address1' , 'latitude' ,'longitude','description' , 'currency' , 'directions' ] , axis =1)

In [31]:
df_main.head(15)

,id_hostel,name,type,totalRatings,facilities,average_rating,count_positive,count_negative,count_neutral
0,102883,Riad Dar Attika,HOTEL,59,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",91.090909,2,3,6
1,102835,Inter-Hotel Le savoy,HOTEL,31,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",88.142857,0,0,7
2,102833,Treasure State Hostel,HOSTEL,323,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",90.011111,9,4,77
3,102800,Onederz Hostel Siem Reap,HOSTEL,4771,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",95.557003,30,41,543
4,102792,Restiny Hostel,HOSTEL,79,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",88.846154,0,2,11
5,102790,Once Again Hostel,HOSTEL,3356,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",92.773994,22,18,283
6,102774,Casa Accademia,GUESTHOUSE,185,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",93.418605,8,13,65
7,102771,Golden Tulip Braga Hotel & SPA,HOTEL,2,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",97.000000,0,0,1
8,102690,The European Place,HOTEL,4,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",64.500000,1,0,1
9,102687,HI Gerês – Pousada de Juventude,HOSTEL,23,"[{'numericId': 1, 'name': 'Free', 'id': 'FACIL...",82.166667,2,1,3


#Making a recomandation system

In [34]:
type_encoder = LabelEncoder()
df_main['type_encoded'] = type_encoder.fit_transform(df_main['type'])


user_preferences = {
    'type': ['HOSTEL' , 'HOTEL' , 'GUESTHOUSE'],
    'facilities': 'Free WiFi, Parking '
}


user_type_encoded = type_encoder.transform(user_preferences['type'])


X = df_main[['type_encoded', 'average_rating', 'count_positive', 'count_negative', 'count_neutral']].values
y = np.random.rand(X.shape[0])


input_layer = Input(shape=(X.shape[1],))
dense1 = Dense(128, activation='relu')(input_layer)
dense2 = Dense(64, activation='relu')(dense1)
output_layer = Dense(1, activation='sigmoid')(dense2)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

model.fit(X, y, epochs=10, batch_size=2, verbose=1)
predictions = model.predict(X)
df_main['predicted_score'] = predictions
top_n_recommendations = df_main.sort_values(by='predicted_score', ascending=False).head(5)
print(top_n_recommendations[['name', 'predicted_score']])


Epoch 1/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - loss: 0.3266
Epoch 2/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.3362
Epoch 3/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 0.3330
Epoch 4/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - loss: 0.3337
Epoch 5/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 0.3348
Epoch 6/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.3369
Epoch 7/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.3326
Epoch 8/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.3309
Epoch 9/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - loss: 0.3350
Epoch 10/10
7324/7324 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.3315
458/458 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
                          name  predicted_score
0              Riad Dar Attika              1.0
9756  Lushy Hostel Nusa Penida              1.0
9758           Sunrise Cabanas              1.0
9759       Hostal Rurrenabaque              1.0
9760             V

#NCF


In [35]:
#Add your preferences
user_interactions = {
    'user_id': [1, 1, 1, 2, 2, 3],
    'rating': [5, 4, 3, 4, 5, 2],
    'type': ['HOTEL', 'HOSTEL', 'GUESTHOUSE', 'HOSTEL', 'HOTEL', 'GUESTHOUSE'],
    'facilities': [
        ['Free WiFi', 'Air Conditioning'],
        ['Free Breakfast', 'Airport Transfers'],
        ['Bar', 'Restaurant'],
        ['Parking', 'Laundry Facilities'],
        ['Free WiFi', 'Free Internet Access'],
        ['Outdoor Terrace', 'Hair Dryers']
    ],
    'average_rating': [91.1, 85.2, 77.0, 88.5, 92.3, 68.9],
}

user_df = pd.DataFrame(user_interactions)
type_encoder = LabelEncoder()
user_df['type_encoded'] = type_encoder.fit_transform(user_df['type'])
mlb = MultiLabelBinarizer()
facilities_encoded = mlb.fit_transform(user_df['facilities'])
X_item = np.hstack((
    user_df[['type_encoded', 'average_rating']].values,
    facilities_encoded
))

#Encode user ids for collaborative filtering
user_encoder = LabelEncoder()
user_df['user_encoded'] = user_encoder.fit_transform(user_df['user_id'])
n_users = user_df['user_encoded'].nunique()
user_input = Input(shape=(1,))
user_embedding = Embedding(n_users, 50)(user_input)
user_flatten = Flatten()(user_embedding)
item_features_input = Input(shape=(X_item.shape[1],))
dense_layer = Dense(128, activation='relu')(item_features_input)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(64, activation='relu')(dense_layer)

combined = Concatenate()([user_flatten, dense_layer])
dense_layer = Dense(64, activation='relu')(combined)
output = Dense(1, activation='sigmoid')(dense_layer)

model = Model(inputs=[user_input, item_features_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
X_user = user_df['user_encoded'].values
X_item_features = X_item
y = user_df['rating'].values
model.fit([X_user, X_item_features], y, epochs=10, batch_size=2, verbose=1)
user_id = 1
user_type_encoded = type_encoder.transform(['HOTEL'])[0]
predictions = model.predict([np.full(len(X_item), user_id), X_item])
user_df['predicted_score'] = predictions
top_n_recommendations = user_df.sort_values(by='predicted_score', ascending=False).head(5)
print(top_n_recommendations[['type', 'predicted_score']])

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 15.2835
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 13.9630  
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 17.4199 
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.4463 
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10.5286 
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.3341 
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.8959 
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.8958  
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 11.2088 
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10.0833 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
         type  predicted_score
0       HOTEL              1.0
1      HOSTEL              1.0
2  GUESTHOUSE              1.0
3      HOSTEL              1.0
4       HOTEL              1.0
